In [1]:
#Load Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#Load Dataset
HousingDB=pd.read_csv('./HousingDB.csv')
HousingDB.head()

,price,lotsize,bedrooms,bathrms,stories,garagepl,recroom,fullbase,airco
0,42000.0,5850,3,1,2,1,no,yes,no
1,38500.0,4000,2,1,1,0,no,no,no
2,49500.0,3060,3,1,1,0,no,no,no
3,60500.0,6650,3,1,2,0,yes,no,no
4,61000.0,6360,2,1,1,0,no,no,no


In [3]:
#Show Key Statistics
HousingDB.describe()

,price,lotsize,bedrooms,bathrms,stories,garagepl
count,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000
mean,68121.597070,5150.265568,2.965201,1.285714,1.807692,0.692308
std,26702.670926,2168.158725,0.737388,0.502158,0.868203,0.861307
min,25000.000000,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,49125.000000,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,62000.000000,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,82000.000000,6360.000000,3.000000,2.000000,2.000000,1.000000
max,190000.000000,16200.000000,6.000000,4.000000,4.000000,3.000000


In [4]:
#Create One Hot Encoded Dataframe

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first')

HousingDB_object = HousingDB.select_dtypes('object')
ohe.fit(HousingDB_object)

codes = ohe.transform(HousingDB_object).toarray()
feature_names = ohe.get_feature_names_out(HousingDB_object.columns)

HousingDB_ohe = pd.concat([HousingDB.select_dtypes(exclude='object'), 
               pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)

HousingDB_ohe.head()

,price,lotsize,bedrooms,bathrms,stories,garagepl,recroom_yes,fullbase_yes,airco_yes
0,42000.0,5850,3,1,2,1,0,1,0
1,38500.0,4000,2,1,1,0,0,0,0
2,49500.0,3060,3,1,1,0,0,0,0
3,60500.0,6650,3,1,2,0,1,0,0
4,61000.0,6360,2,1,1,0,0,0,0


In [5]:
#Define x and y variable
x = HousingDB_ohe.drop(['price'],axis=1).to_numpy()
y = HousingDB_ohe['price'].to_numpy()

#Create Train and Test Datasets
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

#Scale the Data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train2 = sc.fit_transform(x_train)
x_test2 = sc.transform(x_test)

#Model
from sklearn.linear_model import LinearRegression

In [6]:
#Create Standard Model - using Dummy Variables

from sklearn import metrics

for name,method in [('Linear regression', LinearRegression())]: 
    method.fit(x_train2,y_train)
    predict = method.predict(x_test2)

print('\n Regression Model - using OneHotEncoder')
print('\nMethod: {}'.format(name))   

#Coefficents
print('\nIntercept: {:.2f}'.format(float(method.intercept_)))
coeff_table=pd.DataFrame(np.transpose(method.coef_),
                         HousingDB_ohe.drop(['price'],axis=1).columns,
                         columns=['Coefficients'])
print(coeff_table)
    
#R2,MAE,MSE and RMSE
print('\nR2: {:.2f}'.format(metrics.r2_score(y_test,predict)))
adjusted_r_squared = 1-(1-metrics.r2_score(y_test,predict))*(len(y)-1)/(len(y)-x.shape[1]-1)
print('Adj_R2: {:0.2f}'.format(adjusted_r_squared))
print('Mean Absolute Error: {:.2f}'.format(metrics.mean_absolute_error(y_test, predict)))  
print('Mean Squared Error: {:.2f}'.format(metrics.mean_squared_error(y_test, predict)))  
print('Root Mean Squared Error: {:.2f}'.format(np.sqrt(metrics.mean_squared_error(y_test, predict)))) 


 Regression Model - using OneHotEncoder

Method: Linear regression

Intercept: 68860.08
              Coefficients
lotsize        8634.475677
bedrooms       1368.299630
bathrms        7127.520695
stories        6567.390507
garagepl       4597.365623
recroom_yes    2054.126514
fullbase_yes   3661.398670
airco_yes      5736.782817

R2: 0.54
Adj_R2: 0.53
Mean Absolute Error: 12127.85
Mean Squared Error: 253496059.87
Root Mean Squared Error: 15921.56
